In [2]:
# ls -l ./raw_tifs_masks/Point2203_pt1072_31606/TIFs

In [3]:
# read the csv file and mv the files based on the label

In [4]:
import os
root_dir = os.getcwd()
# TODO: replace with your path to the data shared by Candance
csv_labels_file = 'Data/patient_status.csv'
labels_file_path = os.path.join(root_dir,csv_labels_file)
print("labels_file_path is {}".format(labels_file_path))

images_path = os.path.join(root_dir,"Data/raw_tifs_masks")
print("images_path is {}".format(images_path))

labels_file_path is /Users/eilaarich-landkof-stanford/Documents/Code/csiseminar/project/Data/patient_status.csv
images_path is /Users/eilaarich-landkof-stanford/Documents/Code/csiseminar/project/Data/raw_tifs_masks


In [5]:
import pandas as pd

# Read the CSV file using pandas
df = pd.read_csv(os.path.join(root_dir,csv_labels_file),index_col=False)

# Print the DataFrame (optional)
print(df.head())

   PointNumber         Status
0         2203  nonprogressor
1         2204  nonprogressor
2         2206  nonprogressor
3         2302  nonprogressor
4         2303  nonprogressor


In [6]:
df[df['PointNumber'] == 2203]['Status'].values[0]

'nonprogressor'

In [7]:
# collapse tif files to one file
def collapse_tif_files(path_to_files,point_number):
    collpase_file_path = ""
    return collpase_file_path

In [8]:
# organize the images based on labels
list_of_patients_points = os.listdir(images_path)
list_of_patients_points = [x for x in list_of_patients_points if ".DS" not in x and ".ipynb" not in x]
list_of_patients_points[0:5]

['Point3123_pt1162_31688',
 'Point4115_pt1107_31817',
 'Point2313_pt1094_20702',
 'Point4106_pt1162_30997',
 'Point4410_pt1181_31721']

In [9]:
# df[df['PointNumber'] == 4115]['Status'].values[0]

In [10]:
point_dict = {}
for pp in list_of_patients_points:
    label = ""
    point = int(pp.split("_")[0].replace("Point", ""))
    # print("point is {}".format(point))
    # print(type(point))
    fld = os.path.join(images_path,pp)
    # Check if the specific PointNumber is available in the DataFrame
    is_available = point in df['PointNumber'].values
    if is_available is False:
        # print("point {} does not exist".format(point))
        continue
    else:
        label = df[df['PointNumber'] == point]['Status'].values[0]
    point_dict[str(point)] = {"fld":fld,"label":label}

In [11]:
len(point_dict) == 58

True

In [12]:
# create folder for the dataset
dataset_images_path = os.path.join(root_dir,"Data/dataset_images")
print("dataset_images_path is {}".format(dataset_images_path))
os.makedirs(dataset_images_path,exist_ok=True)

dataset_images_path is /Users/eilaarich-landkof-stanford/Documents/Code/csiseminar/project/Data/dataset_images


In [13]:
# get unique list of all the possible tif images 
images_channels_order = []
for point, value in point_dict.items():
    raw_data_path = os.path.join(value['fld'],"TIFs")
    tif_raw_images = os.listdir(raw_data_path)
    # print("tif_raw_images are {}".format(tif_raw_images))
    tif_raw_images = [x.split(".tif")[0] for x in tif_raw_images]
    # print(tif_raw_images)
    images_channels_order.extend(tif_raw_images)
# get unique values
print(len(images_channels_order))
images_channels_order = list(set(images_channels_order))
print(len(images_channels_order))


images_channels_order = [x+".tif" for x in images_channels_order if ".DS_Store" not in x]
# and "SMA" not in x and "PanKRT" not in x]
images_channels_order[0:4]

2727
48


['CK7.tif', 'Ca40.tif', 'CD44.tif', 'Au.tif']

In [15]:
import os
import numpy as np
from PIL import Image

def create_47_channel_image_stack(tif_path, tif_order, output_path,width=1024, height=1024):
    """
    Create a 47-channel image stack from a list of TIFF files.

    Parameters:
        tif_path (str): Path to the directory containing the TIFF images.
        tif_order (list): List of TIFF file names in the desired order.
        width (int): Width of the images (default: 1024).
        height (int): Height of the images (default: 1024).

    Returns:
        None (Saves the multi-page TIFF directly).
    """
    # Number of channels
    channels = len(tif_order)

    # Loop through each channel and save it as a separate image
    for i, tif_file in enumerate(tif_order):
        tif_file_path = os.path.join(tif_path, tif_file)
        if not os.path.isfile(tif_file_path):
            # Channel not found, fill with zeros
            channel_data = np.zeros((height, width), dtype=np.uint8)
        else:
            image_data = Image.open(tif_file_path)

            # Check if the image is already of the desired size (1024x1024)
            if image_data.size != (width, height):
                raise ValueError(f"Image {tif_file} is not of size {width}x{height}")

            # Convert the image to grayscale and store it as a single channel
            channel_data = np.array(image_data)
        # Check if all values in the channel are 0s
        if not np.any(channel_data != 0):
            print(f"!!!Warning: Channel {i+1} is filled with 0s.")

        # Count the number of non-zero pixels in the channel
        num_non_zero_pixels = np.count_nonzero(channel_data)
        # Print the number of non-zero pixels in the channel
        print(f"@@@Channel {i+1}: {num_non_zero_pixels} non-zero pixels out of {width * height}")


        # Save the channel as an image
        channel_image = Image.fromarray(channel_data)
        channel_image_path = f"channel_{i+1}.tif"
        channel_image.save(channel_image_path)

    # Create a list of individual images for multi-page tif
    image_pages = [Image.open(f"channel_{i+1}.tif") for i in range(channels)]

    # print("image_pages is {}. len(image_pages) is {}".format(image_pages,len(image_pages)))
    # print("image_pages[0] is {}".format(image_pages[0]))

    # Save the 47-channel image stack as a multi-page TIFF
    image_pages[0].save(output_path, save_all=True, append_images=image_pages[1:])

    # Remove the temporary channel images
    for i in range(channels):
        os.remove(f"channel_{i+1}.tif")

# # Example usage:
# tif_order = ['1.tif', '2.tif', ..., '47.tif']  # Replace with the list of TIFF file names in the desired order
# tif_path = 'path/to/tifs'  # Replace with the path to the directory containing the TIFF images
# create_47_channel_image_stack(tif_path, tif_order)


In [16]:
!pwd

/Users/eilaarich-landkof-stanford/Documents/Code/csiseminar/project


In [19]:
images_channels_order[0:4]

['CK7.tif', 'Ca40.tif', 'CD44.tif', 'Au.tif']

In [20]:
raw_data_path

'/Users/eilaarich-landkof-stanford/Documents/Code/csiseminar/project/Data/raw_tifs_masks/Point2206_pt1116_31620/TIFs'

In [25]:
len(images_channels_order)

47

In [28]:
# os.path.join(os.getcwd(),"dataset","{}_channels".format(len(images_channels_order)),"labels")

'/Users/eilaarich-landkof-stanford/Documents/Code/csiseminar/project/dataset/47_channels/labels'

In [16]:
# Iterate over key-value pairs in the dictionary
dataset_path=os.path.join(os.getcwd(),"dataset","{}_channels".format(len(images_channels_order)),"labels")
# counter = 0 # for debug
# pil_image_stack=None
for point, value in point_dict.items():
    print(f"Point: {point}, Value: {value}")
    raw_data_path = os.path.join(value['fld'],"TIFs")
    print("raw_data_path is {}".format(raw_data_path))
    label_fld = os.path.join(dataset_path,value['label'])
    print("label_fld is {}".format(label_fld))
    os.makedirs(label_fld,exist_ok=True)
    output_path=os.path.join(label_fld,point+".tif")
    print("output_path is {}".format(output_path))
    create_47_channel_image_stack(raw_data_path, images_channels_order, output_path)
    # create_47_channel_image_stack(raw_data_path, images_channels_order,output_path)
    # for debug
    # counter += 1
    # if counter == 1:
    #     break

Point: 3123, Value: {'fld': '/Users/eilaarich-landkof-stanford/Documents/Code/csiseminar/project/Data/raw_tifs_masks/Point3123_pt1162_31688', 'label': 'progressor'}
raw_data_path is /Users/eilaarich-landkof-stanford/Documents/Code/csiseminar/project/Data/raw_tifs_masks/Point3123_pt1162_31688/TIFs
label_fld is /Users/eilaarich-landkof-stanford/Documents/Code/csiseminar/project/dataset/progressor
output_path is /Users/eilaarich-landkof-stanford/Documents/Code/csiseminar/project/dataset/progressor/3123.tiff
@@@Channel 1: 4857 non-zero pixels out of 1048576
@@@Channel 2: 7134 non-zero pixels out of 1048576
@@@Channel 3: 33762 non-zero pixels out of 1048576
@@@Channel 4: 10153 non-zero pixels out of 1048576
@@@Channel 5: 975509 non-zero pixels out of 1048576
@@@Channel 6: 396 non-zero pixels out of 1048576
@@@Channel 7: 164244 non-zero pixels out of 1048576
@@@Channel 8: 53238 non-zero pixels out of 1048576
@@@Channel 9: 217155 non-zero pixels out of 1048576
@@@Channel 10: 2454 non-zero pix

In [17]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

def read_multi_page_tiff(tiff_path):
    """
    Read a multi-page TIFF image and return a list of NumPy arrays.

    Parameters:
        tiff_path (str): Path to the multi-page TIFF file.

    Returns:
        list: List of NumPy arrays, each representing a channel.
    """
    tiff_image = Image.open(tiff_path)
    num_frames = tiff_image.n_frames

    # Read each channel as a separate NumPy array
    image_channels = [np.array(tiff_image.copy()) for _ in range(num_frames)]

    return image_channels

# # Example usage:
# tiff_path = "47_channel_image_stack.tif"  # Replace with the path to your multi-page TIFF image
# image_channels = read_multi_page_tiff(tiff_path)

# # Set the figure size to display the images larger
# plt.figure(figsize=(20, 15))

# # Display each channel using Matplotlib in multiple rows
# num_rows = 5  # Number of rows
# num_cols = (len(image_channels) + num_rows - 1) // num_rows  # Calculate number of columns

# for i, channel_image in enumerate(image_channels):
#     plt.subplot(num_rows, num_cols, i + 1)
#     plt.imshow(channel_image, cmap='gray')
#     plt.axis('off')
#     plt.title(f"Channel {i + 1}")

# # Enable interactive mode for manual resizing
# plt.ion()
# plt.tight_layout()  # Adjust spacing between subplots
# plt.show()


In [1]:
# import os
# import numpy as np
# from PIL import Image
# from datasets import Dataset, DatasetDict, Features, Array3D

# class CustomTIFFDataset(Dataset):
#     def __init__(self, tif_dir, num_channels=47, width=1024, height=1024):
#         self.tif_dir = tif_dir
#         self.num_channels = num_channels
#         self.width = width
#         self.height = height

#         tif_files = sorted([f for f in os.listdir(tif_dir) if f.startswith('channel_') and f.endswith('.tif')])
#         data = {'image': [], 'image_id': []}

#         for i, tif_file in enumerate(tif_files):
#             image_data = Image.open(os.path.join(tif_dir, tif_file))
#             data['image'].append(np.array(image_data))
#             data['image_id'].append(f'channel_{i+1}')

#         super().__init__(data)

# # Example usage:
# tif_dir = path_to_dataset  # Replace with the path to the directory containing the saved TIFF files
# custom_dataset = CustomTIFFDataset(tif_dir)

# # Convert the custom dataset to a list of dictionaries
# custom_data = [{'image': item['image'], 'image_id': item['image_id']} for item in custom_dataset]

# # Create a dataset dictionary
# dataset_dict = DatasetDict({
#     'train': Dataset.from_dict(custom_data, features=Features({'image': Array3D(shape=(1024, 1024, 47), dtype='uint8')})),
# })

# # Example usage with the loaded custom dataset
# for example in dataset_dict['train']:
#     image_stack = example['image']
#     image_id = example['image_id']
#     print(f"Image ID: {image_id}, Image Stack Shape: {image_stack.shape}")


In [2]:
# # Usage:
# path_to_dataset = "/Users/eilaarich-landkof-stanford/Documents/Code/csiseminar/project/dataset"

# path_to_dataset
